# Group Proposal Group-3

### Introduction

**Pulsar** *(from **pulsa**ting **r**adio source)* or Pulsar Stars are highly magnetic, rotating, compact heavenly bodies often viewed as flickering “stars” from the earth night sky. Pulsar stars belong to the family of neutron stars (or sometimes white dwarfs) that emit beams of electromagnetic radiation from it’s magnetic poles. Due to its high degree of rotation, the electromagnetic radiation from its poles appears to be pulsating or flickering from earth, hence its name.

Pulsars are fantastic cosmic tools for scientists to study a wide range of phenomena. Studying them helps understand a lot of unknown information about the universe and helps us push our understanding of how it works. We mainly detect pulsar stars by studying the radio frequencies received by telescopes. Often radio interferences and random noises interfere and make it hard to detect a pulsar star. Through this project we aim to produce a predictive algorithm which helps us classify if received measurements are from pulsar stars or not. 


### Preliminary exploratory data analysis:

In [1]:
library(tidyverse)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

In [4]:
pulsar <- read_csv("data/pulsar_data_train.csv")

Parsed with column specification:
cols(
  `Mean of the integrated profile` = col_double(),
  `Standard deviation of the integrated profile` = col_double(),
  `Excess kurtosis of the integrated profile` = col_double(),
  `Skewness of the integrated profile` = col_double(),
  `Mean of the DM-SNR curve` = col_double(),
  `Standard deviation of the DM-SNR curve` = col_double(),
  `Excess kurtosis of the DM-SNR curve` = col_double(),
  `Skewness of the DM-SNR curve` = col_double(),
  target_class = col_double()
)



In [5]:
head(pulsar)

Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
121.15625,48.37297,0.3754847,-0.01316549,3.168896,18.39937,7.449874,65.159298,0
76.96875,36.17556,0.7128979,3.38871856,2.399666,17.57100,9.414652,102.722975,0
130.58594,53.22953,0.1334083,-0.29724164,2.743311,22.36255,8.508364,74.031324,0
156.39844,48.86594,-0.2159886,-0.17129365,17.471572,NA,2.958066,7.197842,0
84.80469,36.11766,0.8250128,3.27412537,2.790134,20.61801,8.405008,76.291128,0
121.00781,47.17694,0.2297081,0.09133623,2.036789,NA,9.546051,112.131721,0


In [11]:
colnames(pulsar) <- c("mean_profile", "sd_profile", "kurtosis_profile", "skew_profile", "mean_dmsnr", "sd_dmsnr", "kurtosis_dmsnr", "skew_dmsnr", "target_class")

In [13]:
pulsar <- pulsar %>%
mutate(target_class = as_factor(target_class)) %>%
na.omit(pulsar)
pulsar

mean_profile,sd_profile,kurtosis_profile,skew_profile,mean_dmsnr,sd_dmsnr,kurtosis_dmsnr,skew_dmsnr,target_class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
121.15625,48.37297,0.37548466,-0.01316549,3.1688963,18.399367,7.4498741,65.1592977,0
76.96875,36.17556,0.71289786,3.38871856,2.3996656,17.570997,9.4146523,102.7229747,0
130.58594,53.22953,0.13340829,-0.29724164,2.7433110,22.362553,8.5083638,74.0313242,0
84.80469,36.11766,0.82501279,3.27412537,2.7901338,20.618009,8.4050084,76.2911279,0
109.40625,55.91252,0.56510595,0.05624666,2.7976589,19.496527,9.4432821,97.3745784,0
95.00781,40.21981,0.34757810,1.15316438,2.7700669,18.217741,7.8512050,70.8019377,0
130.08594,46.40262,-0.04658483,-0.03446839,5.0501672,27.997476,6.0598494,38.1547859,0
96.93750,40.53736,0.41940557,0.75582938,1.9715719,11.636964,11.6291195,202.6866400,0
146.69531,47.80618,-0.46841878,0.55492631,2.8854515,18.043781,9.7924207,112.6258021,0


In [6]:
glimpse(pulsar)

Rows: 12,528
Columns: 9
$ `Mean of the integrated profile`               <dbl> 121.15625, 76.96875, 1…
$ `Standard deviation of the integrated profile` <dbl> 48.37297, 36.17556, 53…
$ `Excess kurtosis of the integrated profile`    <dbl> 0.37548466, 0.71289786…
$ `Skewness of the integrated profile`           <dbl> -0.01316549, 3.3887185…
$ `Mean of the DM-SNR curve`                     <dbl> 3.1688963, 2.3996656, …
$ `Standard deviation of the DM-SNR curve`       <dbl> 18.399367, 17.570997, …
$ `Excess kurtosis of the DM-SNR curve`          <dbl> 7.4498741, 9.4146523, …
$ `Skewness of the DM-SNR curve`                 <dbl> 65.1592977, 102.722974…
$ target_class                                   <dbl> 0, 0, 0, 0, 0, 0, 0, 0…


In [10]:
pulsar_split <- initial_split(pulsar, prop = 0.75, strata = target_class)
pulsar_training <- training(pulsar_split)
pulsar_testing <- testing(pulsar_split)

In [11]:
glimpse(pulsar_training)

Rows: 9,396
Columns: 9
$ `Mean of the integrated profile`               <dbl> 76.96875, 156.39844, 8…
$ `Standard deviation of the integrated profile` <dbl> 36.17556, 48.86594, 36…
$ `Excess kurtosis of the integrated profile`    <dbl> 0.71289786, -0.2159886…
$ `Skewness of the integrated profile`           <dbl> 3.38871856, -0.1712936…
$ `Mean of the DM-SNR curve`                     <dbl> 2.3996656, 17.4715719,…
$ `Standard deviation of the DM-SNR curve`       <dbl> 17.570997, NA, 20.6180…
$ `Excess kurtosis of the DM-SNR curve`          <dbl> 9.4146523, 2.9580659, …
$ `Skewness of the DM-SNR curve`                 <dbl> 102.7229747, 7.1978419…
$ target_class                                   <dbl> 0, 0, 0, 0, 0, 0, 0, 0…


### Methods:

### Expected outcomes and significance: